In [2]:
import torch.nn as nn
import torch
from collections import deque
import numpy as np
import torch.optim as optim
import torch.nn.functional as F
import random
import pygame

In [ ]:
class Network(nn.Module):
    def __init__(self,input_states,output_actions,N):
        super().__init__()
        self.m1 = nn.Linear(input_states,N)
        self.m2 = nn.Linear(N,output_actions)
    def forward(self,x):
        x = F.relu(self.m1(x))
        x = self.m2(x)
        return x
Q = Network(len(paddle.state),len(action),10)

optim = optim.Adam(Q.parameters(),lr=0.001)

In [ ]:
episode_number = 100
epsilon = 0.1
batch_size = 1000
gamma = 0.9
for i in range(episode_number):
    #state = [paddlex,paddley,paddle_vel,ballx,bally,ball_velx,ball_vely]
    initial_state = state #for actual program make random initial state

    while True:
        if random.uniform(0,1) < epsilon:
            action = random.choice(action) #replace with actual possible actions
        else:
            action = torch.argmax(Q(torch.tensor([initial_state],dtype=torch.float32))).item()

        next_state, reward = paddle.get_state(initial_state,action)
        memory.append((initial_state,action,reward,next_state))

        if next_state == 0: #fill in with terminating case
            break
        initial_state = next_state

    if len(memory) >= batch_size:
        batch = random.sample(memory,batch_size)

        initial_states = torch.tensor([i[0] for i in batch], dtype=torch.float32)
        actions = torch.tensor([i[1] for i in batch], dtype=torch.float32)
        rewards = torch.tensor([i[2] for i in batch], dtype=torch.float32)
        next_states = torch.tensor([i[3] for i in batch], dtype=torch.float32)

        Q_next = torch.max(Q(next_states), dim=1)[0].unsqueeze(1)

        # Calculate the target Q-values
        target = rewards + gamma * Q_next

        # Calculate the loss
        Q_values = Q(initial_states).gather(1, actions.long().unsqueeze(1))
        loss = nn.MSELoss()(Q_values, target)

        # Backpropagate the error and optimize the Q-network
        optim.zero_grad()
        loss.backward()
        optim.step()


